# Pendulum on Cart System with MPC Control

In this notebook, we will simulate a pendulum on a cart system and control it using a Model Predictive Controller (MPC). The goal is to bring the pendulum from a hanging down position to an upright position.

## 1. Define the Pendulum System

First, we will define the pendulum system with friction and the given model.


In [2]:
import numpy as np
import casadi as ca
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from   openmpc.NonlinearMPC import NonlinearSystem

# Define the pendulum system with friction and given model
def pendulum_system():
    # Pendulum parameters
    m = 0.2
    b = 0.1
    l = 0.3
    I = 0.006
    g = 9.81

    a = m * g * l / (I + m * l**2)
    c = m * l / (I + m * l**2)

    th = ca.MX.sym("th")
    thDot = ca.MX.sym("thDot")
    u = ca.MX.sym("u")  # Control input

    states = ca.vertcat(th, thDot)
    inputs = ca.vertcat(u)

    # Pendulum dynamics with friction term
    rhs = ca.vertcat(
        thDot,                     # dth/dt
        a * ca.sin(th) + c * ca.cos(th) * u - b * thDot  # dth_dot/dt
    )

    return states, inputs, rhs

# Create the NonlinearSystem object
states, inputs, rhs = pendulum_system()
output = states  # Output is the state itself
nlsys_pendulum = NonlinearSystem(updfcn=rhs, states=states, inputs=inputs, outfcn=output)

# Discretize the system
dt = 0.1
nlsys_pendulum_discrete = nlsys_pendulum.c2d(dt)


ModuleNotFoundError: No module named 'openmpc'

## 2. Define the MPC Problem

Next, we will define the MPC problem data, including the prediction model, cost matrices, horizon, and input/output constraints.


In [2]:
# Example mpcProblemData
mpcProblemData = {
    'predictionModel': nlsys_pendulum_discrete,
    'T': 20,
    'Q': np.eye(2),
    'R': np.eye(1),
    'QT': 1000 * np.eye(2),
    'ulim': 10.0,
    'ylim': np.array([np.inf, np.inf]),  # No output constraints
    'dt': dt
}

# Initial state (pendulum hanging down)
x0 = np.array([np.pi, 0])


## 3. Define the MPC Controller Function

We define the function to compute the predicted optimal controls using the MPC framework.


In [8]:
# Define the MPC controller function
def computePredictedOptimalControls(mpcProblemData, x0):
    predictionModel = mpcProblemData['predictionModel']
    f = predictionModel.updfcn; g = predictionModel.outfcn
    n = predictionModel.n; m = predictionModel.m

    # Extract cost matrice and horizon
    Q = mpcProblemData['Q']; R = mpcProblemData['R']; QT = mpcProblemData['QT']
    T = mpcProblemData['T']

    # Extract constraint bounds
    ulim = mpcProblemData['ulim']; ylim = mpcProblemData['ylim']

    # Setup optimization problem and define variables
    opti = ca.Opti()
    X = opti.variable(n, T+1); U = opti.variable(m, T)

    # Define initial state constraint, cost, input/output constraints
    cost = 0
    opti.subject_to(X[:, 0] == x0)
    for t in range(T):
        cost += ca.mtimes([X[:, t].T, Q, X[:, t]])
        cost += ca.mtimes([U[:, t].T, R, U[:, t]])
        opti.subject_to(X[:, t+1] == f(X[:, t], U[:, t]))
        opti.subject_to(opti.bounded(-ylim, g(X[:, t], U[:, t]), ylim))
        opti.subject_to(opti.bounded(-ulim, U[:, t], ulim))
    cost += ca.mtimes([X[:, T].T, QT, X[:, T]])
    opti.minimize(cost)

    # Pick solver (and options), solve problem, and return solution
    opti.solver('ipopt', {"ipopt": {"max_iter": 1000, "print_level": 0}})
    sol = opti.solve()
    return sol.value(X), sol.value(U)

def mpcController(mpcProblemData, x):
    x_sol, u_sol = computePredictedOptimalControls(mpcProblemData, x)
    return u_sol[0]

## 4. Simulate the System with MPC Control

Next, we simulate the pendulum system with the MPC controller to bring it from hanging down to standing up.


In [ ]:
# Simulation parameters
Tsim = 10.0  # Total simulation time
N_steps = int(Tsim / dt)
time = np.linspace(0, Tsim, N_steps + 1)

# Initialize arrays to store the simulation results
x_sim = np.zeros((2, N_steps + 1))
u_sim = np.zeros(N_steps)

# Set initial state
x_sim[:, 0] = x0

# Simulation loop
for k in range(N_steps):
    # Compute the control input using MPC
    u_sim[k] = mpcController(mpcProblemData, x_sim[:, k])
    
    # Apply the control input to the system
    x_sim[:, k + 1] = nlsys_pendulum_discrete.updfcn(x_sim[:, k], u_sim[k]).full().flatten()

## 5. Plot the results

Finally, we plot the results.


In [ ]:
# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Plot the state trajectories on the first subplot
ax1.plot(time, x_sim[0, :], label='theta', linewidth=2)
ax1.plot(time, x_sim[1, :], label='theta_dot', linewidth=2)
ax1.set_xlabel('Time [s]', fontsize=18)
ax1.set_ylabel('State', fontsize=18)
ax1.set_xlim([0, Tsim])
ax1.set_ylim([-2.5 * np.pi, 2.5 * np.pi])
ax1.legend(fontsize=14)
ax1.set_title('State Trajectories', fontsize=18)
ax1.grid(True)

# Set grid steps
ax1.xaxis.set_major_locator(ticker.MultipleLocator(1.0))
ax1.yaxis.set_major_locator(ticker.MultipleLocator(np.pi / 4))

# Plot the control trajectory on the second subplot
ax2.step(time[:-1], u_sim, 'k', label='u', where='post', linewidth=2)
ax2.set_xlabel('Time [s]', fontsize=18)
ax2.set_ylabel('Control', fontsize=18)
ax2.set_xlim([0, Tsim])
ax2.set_ylim([-10, 10])
ax2.legend(fontsize=14)
ax2.set_title('Control Trajectory', fontsize=18)
ax2.grid(True)

# Set grid steps
ax2.xaxis.set_major_locator(ticker.MultipleLocator(1.0))
ax2.yaxis.set_major_locator(ticker.MultipleLocator(2))

# Set tick font size
ax1.tick_params(axis='both', which='major', labelsize=14)
ax2.tick_params(axis='both', which='major', labelsize=14)

# Adjust layout to make space for larger labels
plt.tight_layout(pad=3.0)

# Save the plot as a PDF file
plt.savefig('Figures/mpc_simulation_pendulum.pdf', format='pdf')

# Show the plots
plt.show()